# Assignment 2: Milestone I Natural Language Processing
## Task 2&3
#### Student Name: Jidhin Poulose
#### Student ID: s4023503


Environment: Python 3 and Jupyter notebook

Libraries used: please include all the libraries you used in your assignment, e.g.,:
* pandas
* re
* numpy

## Introduction
You should give a brief information of this assessment task here.

<span style="color: red"> Note that this is a sample notebook only. You will need to fill in the proper markdown and code blocks. You might also want to make necessary changes to the structure to meet your own needs. Note also that any generic comments written in this notebook are to be removed and replace with your own words.</span>

## Importing libraries

In [ ]:
# Code to import libraries as you need in this assessment, e.g.,
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import KeyedVectors
from collections import defaultdict

In [ ]:
# Read the csv file
data = pd.read_csv('processed.csv')

In [ ]:
data.head(5)

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,Processed_Review
0,1077,60,Some major design flaws,i had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,high hopes wanted work initially petite usual ...
1,1049,50,My favorite buy!,"i love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,jumpsuit fun flirty fabulous time compliments
2,847,47,Flattering shirt,this shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,shirt due adjustable front tie length leggings...
3,1080,49,Not for the very petite,"i love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses,tracy reese dresses petite feet tall brand pre...
4,858,39,Cagrcoal shimmer fun,i aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits,basket hte person store pick teh pale hte gorg...


In [ ]:
data.isnull().sum()

,0
Clothing ID,0
Age,0
Title,0
Review Text,0
Rating,0
Recommended IND,0
Positive Feedback Count,0
Division Name,0
Department Name,0
Class Name,0


In [ ]:
data = data.dropna()

In [ ]:
data.isnull().sum()

,0
Clothing ID,0
Age,0
Title,0
Review Text,0
Rating,0
Recommended IND,0
Positive Feedback Count,0
Division Name,0
Department Name,0
Class Name,0


In [ ]:
tk_reviews = data['Processed_Review']

In [ ]:
tk_reviews = tk_reviews.to_list()

In [ ]:
tk_reviews[1:3]

['jumpsuit fun flirty fabulous time compliments',
 'shirt due adjustable front tie length leggings sleeveless pairs cardigan shirt']

In [ ]:
type(tk_reviews)

list

In [ ]:
tk_reviews = [review.split(" ") for review in tk_reviews if review]


In [ ]:
tk_reviews[1:3]

[['jumpsuit', 'fun', 'flirty', 'fabulous', 'time', 'compliments'],
 ['shirt',
  'due',
  'adjustable',
  'front',
  'tie',
  'length',
  'leggings',
  'sleeveless',
  'pairs',
  'cardigan',
  'shirt']]

In [ ]:
# Load the vocabulary
def load_vocab(vocab_file):
    with open(vocab_file, 'r') as f:
        vocab = {line.strip().split(':')[0]: int(line.strip().split(':')[1]) for line in f}
    return vocab

In [ ]:
vocab_file = 'vocab.txt'
vocab = load_vocab(vocab_file)

## Task 2. Generating Feature Representations for Clothing Items Reviews

In [ ]:
# binding the words together for each review
joined_reviews = [' '.join(review) for review in tk_reviews]

In [ ]:
import os
if not os.path.exists('glove.6B.zip'):
    print("Downloading GloVe embeddings...")
    !wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip -o -q glove.6B.zip -d glove

--2024-10-01 04:47:59--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-10-01 04:47:59--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M   885KB/s    in 16m 26s 

2024-10-01 05:04:26 (854 KB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [ ]:
# Open the GloVe pre-trained embeddings file
from gensim.models import KeyedVectors
with open('glove/glove.6B.50d.txt') as f:
    keys, weights = [], []
    for l in f:
      # Split each line into the word and its vector representation
      k, v = l.split(maxsplit=1)
      keys.append(k)
      # Convert the vector string to a numpy array and add to weights list
      weights.append(np.fromstring(v, sep=' '))
# Create a KeyedVectors object to store the embeddings
# The vector size is determined by the shape of the first weight vector
glove_embeddings = KeyedVectors(weights[0].shape[0], count=len(keys))
glove_embeddings.add_vectors(keys, weights)
preTGloVe_wv = glove_embeddings

In [ ]:
print("Found %s word vectors." % len(preTGloVe_wv))

Found 800000 word vectors.


### **Count vectors representation**

In [ ]:
# Initialize CountVectorizer with the existing vocabulary
cVectorizer = CountVectorizer(analyzer="word", vocabulary=vocab)
count_features = cVectorizer.fit_transform(joined_reviews)  # Generate count features
count_features.shape

(19652, 7529)

In [ ]:
# Function to write vector file (modified to include webindex)
def write_vectorFile_with_index(data_features, filename, webindices):
  # Get the number of rows (documents) in the data_features matrix
  num = data_features.shape[0]
  with open(filename, 'w') as out_file:
    # Iterate through each row (document) in the data_features matrix
    for a_ind in range(0, num):
        out_file.write(f"#{webindices[a_ind]},")
        # Iterate through non-zero elements in the current row
        # This is efficient for sparse matrices
        for f_ind in data_features[a_ind].nonzero()[1]:
          # Get the value at the current position and convert to int
          value = int(data_features[a_ind, f_ind])
          out_file.write(f"{f_ind}:{value},")
        out_file.write('\n')

### **Unweighted (binary) vector representation using GloVe**

In [ ]:
# Initialize CountVectorizer with binary=True
bVectorizer = CountVectorizer(analyzer="word", binary=True, vocabulary=vocab)
# Generate binary features
binary_features = bVectorizer.fit_transform(joined_reviews)
binary_features.shape

(19652, 7529)

In [ ]:
# Function to generate unweighted document vectors
def unweighted_docvecs(embeddings, binary_features, docs):
  # Initialize a zero matrix to store document vectors
  vecs = np.zeros((len(docs), embeddings.vector_size))
  for i, doc in enumerate(docs):
    # Filter out terms that are not in the embedding vocabulary
    valid_keys = [term for term in doc if term in embeddings.key_to_index]
    # Create binary weights: 1 if the term is present in the document, 0 otherwise
    binary_weights = [1 if binary_features[i, vocab[term]] > 0 else 0 for term in valid_keys]
    # Multiply each term's embedding by its binary weight
    weighted = [embeddings[term] * w for term, w in zip(valid_keys, binary_weights)]
    # Sum up all the weighted embeddings to get the document vector
    docvec = np.sum(weighted, axis=0)
    # Store the document vector in the results matrix
    vecs[i,:] = docvec
  return vecs

In [ ]:
# Generate unweighted document vectors
unweighted_preTGloVe_dvs = unweighted_docvecs(preTGloVe_wv, binary_features, tk_reviews)

In [ ]:
unweighted_preTGloVe_dvs

array([[ 3.57157302, -3.96359205,  3.31884074, ..., -0.23908034,
         2.97417307, -2.44539618],
       [-2.34051514,  1.39787209, -4.70805264, ..., -1.65546536,
         0.38116995,  1.65550005],
       [-5.53618002,  3.77994132, -2.72567987, ..., -6.70081997,
        -1.06155181, -4.32599688],
       ...,
       [ 1.12529898, -0.69280398,  0.26605999, ..., -1.41801202,
        -0.11023009,  2.23825979],
       [ 2.91901064,  3.47800589,  1.67871833, ..., -4.15545177,
        -2.15171194,  2.47598886],
       [ 1.30052102,  0.42059994, -5.66381025, ..., -0.34029651,
         1.61652899,  3.62967396]])

### **TF-IDF Vector Represenation**

In [ ]:
def gen_vocIndex(voc_fname):
  with open(voc_fname) as vocf:
    voc_Ind = [l.split(':') for l in vocf.read().splitlines()]
  return {int(vi[1]):vi[0] for vi in voc_Ind}

# Generates the w_index:word dictionary
voc_fname = './vocab.txt'
voc_dict = gen_vocIndex(voc_fname)
voc_dict

{0: 'a-cup',
 1: 'a-flutter',
 2: 'a-frame',
 3: 'a-kind',
 4: 'a-line',
 5: 'a-lines',
 6: 'a-symmetric',
 7: 'aa',
 8: 'ab',
 9: 'abbey',
 10: 'abby',
 11: 'abdomen',
 12: 'ability',
 13: 'abnormally',
 14: 'abo',
 15: 'abou',
 16: 'above-the',
 17: 'abroad',
 18: 'abs',
 19: 'absolute',
 20: 'absolutely',
 21: 'absolutley',
 22: 'absolutly',
 23: 'abstract',
 24: 'absurd',
 25: 'abt',
 26: 'abundance',
 27: 'ac',
 28: 'accent',
 29: 'accented',
 30: 'accenting',
 31: 'accents',
 32: 'accentuate',
 33: 'accentuated',
 34: 'accentuates',
 35: 'accentuating',
 36: 'accept',
 37: 'acceptable',
 38: 'accepted',
 39: 'access',
 40: 'accessories',
 41: 'accessorize',
 42: 'accessorized',
 43: 'accessorizing',
 44: 'accessory',
 45: 'accident',
 46: 'accidental',
 47: 'accidentally',
 48: 'accommodate',
 49: 'accommodated',
 50: 'accommodates',
 51: 'accommodating',
 52: 'accomodate',
 53: 'accompanying',
 54: 'accomplish',
 55: 'accordian',
 56: 'account',
 57: 'accurate',
 58: 'accurately

In [ ]:
# Function to create a TF-IDF representation of thee review
from sklearn.feature_extraction.text import TfidfVectorizer
tVectorizer = TfidfVectorizer(analyzer = "word",vocabulary = vocab) # initialised the TfidfVectorizer
tfidf_features = tVectorizer.fit_transform(joined_reviews) # generate the tfidf vector representation for all articles
tfidf_features.shape

(19652, 7529)

In [ ]:
def write_vectorFile(data_features,filename):
    num = data_features.shape[0] # the number of document
    out_file = open(filename, 'w') # creates a txt file and open to save the vector representation
    for a_ind in range(0, num): # loop through each article by index
        for f_ind in data_features[a_ind].nonzero()[1]: # for each word index that has non-zero entry in the data_feature
            value = data_features[a_ind][0,f_ind] # retrieve the value of the entry from data_features
            out_file.write("{}:{} ".format(f_ind,value)) # write the entry to the file in the format of word_index:value
        out_file.write('\n') # start a new line after each article
    out_file.close() # close the file

In [ ]:
tVector_file = "./review_tVector.txt" # file name of the tfidf vector
write_vectorFile(tfidf_features,tVector_file) # write the tfidf vector to file

In [ ]:
def doc_wordweights (fName_tVectors, voc_dict):
  tfidf_weights = []

  with open(fName_tVectors) as tVecf:
    tVectors = tVecf.read().splitlines()
  for tv in tVectors: # for each tfidf document vector
    tv = tv.strip()
    weights = tv.split(' ') # list of 'word_index:weight' entries
    weights = [w.split(':') for w in weights] # list of [word_index, weight] entries
    wordweight_dict = {voc_dict[int(w[0])]: w[1] for w in weights} # dictionary of word:weight pairs
    tfidf_weights.append(wordweight_dict)
  return tfidf_weights

fName_tVectors = 'review_tVector.txt' # file name of the tfidf vector
tfidf_weights = doc_wordweights(fName_tVectors, voc_dict)

In [ ]:
tfidf_weights[0]

{'bottom': '0.1049090504878116',
 'cheap': '0.1383468173355121',
 'design': '0.09819053319703584',
 'directly': '0.22042366265962404',
 'fact': '0.1468703738136722',
 'flaw': '0.18693685782734779',
 'found': '0.11016820762903572',
 'half': '0.29992142257858667',
 'high': '0.10509713403953783',
 'hopes': '0.16844705121993464',
 'imo': '0.19309847172209632',
 'initially': '0.1709779316124002',
 'layer': '0.27389678699967074',
 'layers': '0.1640176063766487',
 'major': '0.19468911285051196',
 'medium': '0.09045659947118839',
 'net': '0.4867876227942156',
 'nicely': '0.11400095142158767',
 'outrageously': '0.2567966207221876',
 'petite': '0.17993461419688878',
 'reordered': '0.20116611478729474',
 'sewn': '0.15731076193390783',
 'tight': '0.09944270371682067',
 'usual': '0.11089799670931574',
 'wanted': '0.10688230992054927',
 'work': '0.08942754431903752',
 'zip': '0.1579093630253308',
 'zipper': '0.1379665564316224'}

In [ ]:
def weighted_docvecs(embeddings, tfidf, docs):
    # Initialize a zero matrix to store document vectors
    # Shape: (number of documents, embedding dimension)
    vecs = np.zeros((len(docs), embeddings.vector_size))

    # Iterate through each document
    for i, doc in enumerate(docs):
        # Filter out terms that are not in the embedding vocabulary
        valid_keys = [term for term in doc if term in embeddings.key_to_index]

        # Get TF-IDF weights for valid terms
        # If a term is not in the TF-IDF matrix, its weight is set to 0
        tf_weights = [float(tfidf[i].get(term, 0.)) for term in valid_keys]

        # Ensure that we have the same number of terms and weights
        assert len(valid_keys) == len(tf_weights)

        # Multiply each term's embedding by its TF-IDF weight
        weighted = [embeddings[term] * w for term, w in zip(valid_keys, tf_weights)]

        # Stack the weighted embeddings vertically
        docvec = np.vstack(weighted)

        # Sum up all the weighted embeddings to get the document vector
        docvec = np.sum(docvec, axis=0)

        # Store the document vector in the results matrix
        vecs[i,:] = docvec

    return vecs

In [ ]:
def weighted_docvecs(embeddings, tfidf, docs):
    # Initialize a zero matrix to store document vectors
    vecs = np.zeros((len(docs), embeddings.vector_size))
    for i, doc in enumerate(docs):
        # Filter out terms that are not in the embedding vocabulary
        valid_keys = [term for term in doc if term in embeddings.key_to_index]
        #  Get TF-IDF weights for valid terms
        # If a term is not in the TF-IDF matrix, its weight is set to 0
        tf_weights = [float(tfidf[i].get(term, 0.)) for term in valid_keys]
        assert len(valid_keys) == len(tf_weights)
        # Multiply each term's embedding by its TF-IDF weight
        weighted = [embeddings[term] * w for term, w in zip(valid_keys, tf_weights)]
        # Stack the weighted embeddings vertically
        docvec = np.vstack(weighted)
        # Sum up all the weighted embeddings to get the document vector
        docvec = np.sum(docvec, axis=0)
        vecs[i,:] = docvec
    return vecs

In [ ]:
weighted_preTGloVe_dvs = weighted_docvecs(preTGloVe_wv, tfidf_weights, tk_reviews)

In [ ]:
weighted_preTGloVe_dvs

array([[ 0.81326997, -1.20487916,  1.61317408, ...,  0.35037398,
         0.98145258, -0.72741616],
       [-1.02930331,  0.49924228, -2.1061604 , ..., -0.78494704,
         0.23018403,  0.64478934],
       [-1.99558759,  1.09937561, -0.89829326, ..., -2.16219521,
        -0.19498554, -1.36913705],
       ...,
       [ 0.44179174, -0.32263976, -0.0057269 , ..., -0.63758403,
        -0.1635806 ,  1.01626301],
       [ 0.66972482,  0.61471367,  0.64147234, ..., -0.52802664,
        -0.41661435,  0.86121458],
       [ 0.47965372,  0.20797431, -2.14299273, ..., -0.08895668,
         0.61969817,  1.32442343]])

### **Saving outputs**
Save the count vector
- count_vectors.txt

In [ ]:
# Create webindices from Clothing ID
webindices = data['Clothing ID'].tolist()

In [ ]:
# Write count vectors to file
count_vector_file = "./count_vectors.txt"
write_vectorFile_with_index(count_features, count_vector_file, webindices)

In [ ]:
def read_count_vectors(file_path, num_samples=5):
    with open(file_path, 'r') as file:
        lines = file.readlines()[:num_samples]  # Read only the first 5 lines

    print(f"First {num_samples} lines from {file_path}:")
    for line in lines:
        print(line.strip())

read_count_vectors('count_vectors.txt')

First 5 lines from count_vectors.txt:
#1077,687:1,1028:1,1716:1,1792:1,2289:1,2481:1,2602:1,2892:2,3010:1,3087:1,3193:1,3258:1,3549:2,3552:1,3832:1,3934:1,4224:2,4234:1,4427:1,4639:2,5260:1,5668:1,6726:1,7092:1,7207:1,7406:1,7520:1,7522:1,
#1049,1287:1,2284:1,2502:1,2667:1,3403:1,6739:1,
#847,86:1,925:1,1988:1,2646:1,3584:1,3595:1,4506:1,5736:2,5924:1,6716:1,
#1080,179:1,721:1,1950:1,2083:1,2373:1,2610:1,2657:1,2711:1,3168:1,3707:1,3748:1,4472:1,4484:1,4639:1,4912:1,5176:1,5332:1,5764:1,5900:2,6290:1,6366:1,6548:1,6809:1,7406:1,
#858,408:1,449:1,818:1,1630:1,2050:1,2803:1,3120:4,4363:1,4513:1,4528:1,4628:1,4639:1,4663:1,4883:1,5903:1,6274:1,6601:1,6907:1,7469:1,


## Task 3. Clothing Review Classification

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from scipy.sparse import issparse

In [ ]:
# Prepare the target variable
y = data['Recommended IND'].values

In [ ]:
# Function to perform classification and cross-validation
def classify_and_evaluate(X, y):
    if issparse(X):
        # For sparse matrices, we don't need to scale
        model = LogisticRegression(random_state=42)
    else:
        # For dense matrices, we can use StandardScaler
        scaler = StandardScaler(with_mean=not issparse(X))
        X = scaler.fit_transform(X)
        model = LogisticRegression(random_state=42)

    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    return np.mean(scores)

### **Language model comparisons**

In [ ]:
# Count vectors
count_accuracy = classify_and_evaluate(count_features, y)

# Binary vectors
binary_accuracy = classify_and_evaluate(binary_features, y)

# TF-IDF vectors
tfidf_accuracy = classify_and_evaluate(tfidf_features, y)

# Weighted GloVe vectors
weighted_glove_accuracy = classify_and_evaluate(weighted_preTGloVe_dvs, y)

# Unweighted GloVe vectors
unweighted_glove_accuracy = classify_and_evaluate(unweighted_preTGloVe_dvs, y)

print("Q1 Results:")
print(f"Count Vectors Accuracy: {count_accuracy:.4f}")
print(f"Binary Vectors Accuracy: {binary_accuracy:.4f}")
print(f"TF-IDF Vectors Accuracy: {tfidf_accuracy:.4f}")
print(f"Weighted Glove Vector Accuracy: {weighted_glove_accuracy:.4f}")
print(f"Unweighted GloVe Vectors Accuracy: {unweighted_glove_accuracy:.4f}")

# Q2: Checking if better accuracy can be achieved.

# Using TF-IDF vectorizer for this example
vectorizer = TfidfVectorizer(analyzer="word", vocabulary=vocab)

# Only title
title_features = vectorizer.fit_transform(data['Title'])
title_accuracy = classify_and_evaluate(title_features, y)

# Only review text (already done in Q1 with tfidf_accuracy)

# Both title and review text
combined_text = data['Title'] + " " + data['Review Text']
combined_features = vectorizer.fit_transform(combined_text)
combined_accuracy = classify_and_evaluate(combined_features, y)

print("\nQ2 Results:")
print(f"Title Only Accuracy: {title_accuracy:.4f}")
print(f"Review Text Only Accuracy: {tfidf_accuracy:.4f}")
print(f"Combined Title and Review Text Accuracy: {combined_accuracy:.4f}")

Q1 Results:
Count Vectors Accuracy: 0.8753
Binary Vectors Accuracy: 0.8759
TF-IDF Vectors Accuracy: 0.8737
Weighted Glove Vector Accuracy: 0.8257
Unweighted GloVe Vectors Accuracy: 0.8255

Q2 Results:
Title Only Accuracy: 0.8623
Review Text Only Accuracy: 0.8737
Combined Title and Review Text Accuracy: 0.8867


## Summary

* Simpler bag-of-words models outperformed complex representations for this task  
* Combining title and review text improved classification accuracy  
* High accuracies (>82%) indicate strong predictive signals in the review text  
* Feature engineering and multiple information sources are crucial for NLP tasks in e-commerce applications  

## Reference  

I have used the functions from the follwoing activities notebook files provided in the RMIT Canvas.  
* w07_act1_gen_feat_vec.ipynb
* w07_act2_classification.ipynb
* w08_act1_term_embedding.ipynb
* w08_act2_embedding_classification.ipynb
